<a href="https://colab.research.google.com/github/HelenNunez/BALG_Project3/blob/master/Model_Test_Laura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install findspark
# # Start a SparkSession

# # Install Java, Spark, and Findspark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark



Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp /content/drive/My\ Drive/BALG_GoogleDriveLN -r /content/ 

cp: cannot open '/content/drive/My Drive/BALG_GoogleDriveLN/trump_sentiment_new.gsheet' for reading: Operation not supported
cp: cannot open '/content/drive/My Drive/BALG_GoogleDriveLN/sentiment_sentiment_new.gsheet' for reading: Operation not supported
cp: cannot open '/content/drive/My Drive/BALG_GoogleDriveLN/biden_sentiment_For_SR (1).csv/positive biden.gsheet' for reading: Operation not supported


In [4]:
import os
import time
import datetime
import glob
import pandas as pd
import findspark
import json

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

findspark.init()
import pyspark

from pyspark import SparkContext, SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StringType, StructType, ArrayType
from pyspark.sql.functions import format_number as fmt
from pyspark.ml.feature import Word2Vec, Word2VecModel
from pyspark.sql.functions import udf, avg, col


from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob

In [5]:
# Read Trump and Biden Data File and store into Pandas Data Frames
trump_data = pd.read_csv('./BALG_GoogleDriveLN/trump_reddit_pull_test.csv')
biden_data = pd.read_csv('./BALG_GoogleDriveLN/biden_reddit_pull_test.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
trump_data.head()
trump_data.tail()

,Unnamed: 0,associated_award,author,author_fullname,author_premium,awarders,body,collapsed_because_crowd_control,created_utc,gildings,id,is_submitter,link_id,locked,no_follow,parent_id,permalink,retrieved_on,score,send_replies,stickied,subreddit,subreddit_id,total_awards_received,created,edited,author_cakeday,distinguished,treatment_tags
84735,84735,NaN,AutoModerator,t2_6l4z3,True,[],\nThis subreddit is a pro-Trump subreddit for ...,NaN,1598141044,{},g2j85bk,False,t3_ietdnt,False,True,t3_ietdnt,/r/trump/comments/ietdnt/snopes_once_again_dec...,1598145630,1,False,False,trump,t5_2r18q,0,1.598141e+09,NaN,NaN,moderator,[]
84736,84736,NaN,ConsistentPayment,t2_28k5wtcf,False,[],&gt; If people think all politicians are corru...,NaN,1598141017,{},g2j83pu,False,t3_iervb6,False,False,t1_g2j3y5u,/r/preppers/comments/iervb6/winter_is_coming/g...,1598145608,11,True,False,preppers,t5_2riow,0,1.598141e+09,NaN,NaN,NaN,[]
84737,84737,NaN,AutoModerator,t2_6l4z3,True,[],\nThis subreddit is a pro-Trump subreddit for ...,NaN,1598140981,{},g2j81ge,False,t3_ietd20,False,True,t3_ietd20,/r/trump/comments/ietd20/and_the_left_calls_tr...,1598145575,1,False,False,trump,t5_2r18q,0,1.598141e+09,NaN,NaN,moderator,[]
84738,84738,NaN,AutoModerator,t2_6l4z3,True,[],\nThis subreddit is a pro-Trump subreddit for ...,NaN,1598140942,{},g2j7z4m,False,t3_ietcqe,False,True,t3_ietcqe,/r/trump/comments/ietcqe/this_guy_gets_it/g2j7...,1598145544,1,False,False,trump,t5_2r18q,0,1.598141e+09,NaN,NaN,moderator,[]
84739,84739,NaN,technicallycorrect2,t2_14lcb9,True,[],I heard donald trump was going to rake the pin...,NaN,1598140924,{},g2j7y15,False,t3_ies3bd,False,True,t1_g2j7s96,/r/news/comments/ies3bd/california_wildfires_a...,1598145528,-1,True,False,news,t5_2qh3l,0,1.598141e+09,NaN,NaN,NaN,[]


In [7]:
biden_data['date'] = biden_data['retrieved_on'].apply(lambda x:datetime.datetime.fromtimestamp(x))  
#New_Data['Date']

In [8]:
biden_data['just_date'] = biden_data['date'].dt.date

In [9]:
#Combine the data into a single dataset
biden_body_df = biden_data[['date', 'just_date','body']] 

# Start Spark Session

In [10]:
os.getcwd()

'/content'

In [18]:
sc = SparkContext("local", "Model")

In [19]:
sc._conf.getAll()

[('spark.master', 'local'),
 ('spark.app.name', 'Model'),
 ('spark.app.id', 'local-1598905348254'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.port', '38561'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', 'b3b963fc9d83'),
 ('spark.ui.showConsoleProgress', 'true')]

In [23]:
SparkContext.setSystemProperty('spark.executor.memory','70g')
SparkContext.setSystemProperty('spark.driver.memory','70g')
SparkContext.setSystemProperty('spark.driver.maxResultSize','0')
sc._conf.getAll()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/content/spark-2.4.6-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-2.4.6-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/content/spark-2.4.6-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40443)
Traceback (most recent call last):
  File "/content/spark-2.

Py4JNetworkError: ignored

In [20]:
#appName = "Model"
#master = "local"
#spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

In [21]:
#SparkContext.stop(sc)

In [22]:
model = Word2VecModel.load('./BALG_GoogleDriveLN/Models/one_million_twoptone')

Py4JJavaError: ignored

In [ ]:
diction = model.getVectors()

In [16]:
biden_df = spark.createDataFrame(biden_body_df)

NameError: ignored

In [ ]:
cosine_similarity([[1,0,-1]],[[-1, -1, 0]])

array([[-0.5]])

In [ ]:
test = "goodness"

In [ ]:
test = ["trump is so great. I love him", "Joe Biden for the Win"]

In [ ]:
diction.collect()[diction["word"] == "alligator"]["vector"]

TypeError: list indices must be integers or slices, not Column

In [ ]:
diction.first().word

'cristino'

In [ ]:
diction.first().vector

DenseVector([-0.1161, -0.1775, 0.1231, -0.0801, 0.0813, 0.1487, -0.0536, 0.2583, -0.0229, 0.0011, -0.2627, 0.0232, 0.0512, -0.0504, -0.033, -0.0342, 0.088, 0.095, -0.0076, 0.0302, 0.0347, -0.0656, 0.0552, 0.1039, -0.1076, 0.0647, -0.0177, -0.0108, 0.1604, 0.1316, -0.0866, -0.0141, 0.0339, -0.0128, 0.0561, -0.0103, -0.0601, -0.0728, 0.0823, -0.0641, 0.0555, -0.1415, -0.1054, -0.0805, 0.1119, -0.0278, -0.0534, -0.0268, 0.0846, 0.1385, -0.0235, -0.1821, -0.1792, -0.0029, 0.2352, 0.0786, -0.242, -0.0798, -0.1995, 0.0964, -0.074, -0.1611, -0.0533, 0.121, 0.1041, -0.0404, 0.157, 0.1125, 0.0538, 0.0526, -0.0637, -0.0131, 0.0377, 0.0803, 0.1078, 0.0374, -0.0204, -0.1541, 0.0511, -0.014, -0.1813, 0.0673, 0.1497, -0.0247, 0.1128, 0.0413, 0.0873, -0.0177, -0.0465, 0.0495, -0.0844, -0.037, -0.0349, 0.0561, 0.1187, -0.09, -0.0318, 0.1475, -0.0955, -0.08, -0.0762, 0.0489, 0.0807, 0.0814, 0.0452, 0.0237, 0.1183, -0.0671, -0.1631, -0.0766, -0.0991, 0.0876, 0.0043, 0.0337, 0.0365, -0.0859, 0.0742, 0.03

In [ ]:
word = diction.filter(diction.word == "batman")["vector"]

In [ ]:
sent = ("This is text I want to know about").split(" ")
                                                   
                                
doc = spark.createDataFrame([(sent,), (sent,)],["sentence"])


In [ ]:
type(doc)

pyspark.sql.dataframe.DataFrame

In [ ]:
worthing = diction.filter(diction.word == "batman")

In [ ]:
model.transform(doc).head().model

AttributeError: 'NoneType' object has no attribute '_jvm'

In [ ]:
vec = worthing.collect()[0].vector

In [ ]:
cosine_similarity([vec], [vec])
#vector representing batman is 100 percent similar to itself.

array([[1.]])

In [ ]:
model.getVectors().show()

+-------------+--------------------+
|         word|              vector|
+-------------+--------------------+
|     cristino|[-0.1160977184772...|
|    hoshizora|[-0.1109780073165...|
|        mells|[-0.0570699460804...|
|     17901791|[-0.0185965653508...|
|        frane|[-0.1329791694879...|
|        gawar|[-0.0898746177554...|
|          kuş|[-0.0321204923093...|
|        toppo|[0.00813362747430...|
|        shuvo|[-0.1562451720237...|
|       hilyer|[-0.0437848530709...|
|        cetti|[0.13472637534141...|
|         koel|[-0.0291677471250...|
|        10292|[0.03037112019956...|
|rhynchostylis|[0.09231655299663...|
|        1910s|[0.11900905519723...|
|        19125|[0.00824300106614...|
|         onam|[-0.0484162457287...|
|      rahmani|[-0.2499719411134...|
|      prinzip|[-0.0082170348614...|
|       197077|[-0.0509997271001...|
+-------------+--------------------+
only showing top 20 rows



either need to write a function that gets the dense vector for each word,and averages a list of words pairwise.

numpy or linear alg in sklearn

transform should do that already---eric's understanding is default behavior is it does the averaging for us--we tokenize sentence


goa;" vector representatiopm pf a reddit comment. that vec rep is the average of all the vectors for the words--removing stopwords, probably :)

once we have ths, what do we do?


how similar is our vector of a reddit comment to a vector of "leader" or "winner"? (and do it against negative words)
--could be like sentiment analysis

if we had positive/negative classifications already, we could run those thru--single point for the average of negative or positive

we can compare differences
we can find synonyms--metaphors

can we identify the word that differentiates biden from trump? 

a vector is a direction...anda word

find the vector for trump, and for biden. then with the vector we try to find the word


look for themes around how people talk about each person.

take single comment, create vector, look for synonyms
rice, repeat.---count

In [ ]:
model.findSynonyms("doctor", 6).select("word", fmt("similarity", 5).alias("similarity")).show()

AttributeError: 'NoneType' object has no attribute '_jvm'